# Stand-Alone Self-Attention in Vision Models Summary
> Introducing Self-Attention as a stand-alone layer in Convolutional Architectures by Prajit Ramachandran,Niki Parmar,Ashish Vaswani,Jonathon Shlens,Anselm Levskaya,Irwan Bello

- toc: true 
- badges: false
- comments: true
- categories: [jupyter]
- image: images/SATCONV.png

[Paper Link](https://arxiv.org/pdf/1906.05909.pdf)
## What did the authors want to achieve ?   
Combat the deficits of Conolutional Architectures, (local connectivity, failing to reason globally) by introducing Attention to as a stand-alone layer. The authors prove that this can be both more accurate and more efficient at the same time. Architectures that are attention only, and a mixed version of convolutional and attention architectures are introduced and compared to the vanilla convolutional implementations. 


## Methods
<br/><br/>
### Conv Block deficits 


### Self-Attention 

#### Replacing Spatial Convolutions 

#### Replacing the Convolutional Stem (intial layers of the CNN) 

## Results
